In [ ]:
import numpy as np
from mst_algorithm import MST
from mst_graphs_local import NetworkXGraph
import networkx as nx
import random
import time


In [ ]:
global_size = 100
global_degree = 4
global_max_weight = 10
global_max_read = 300

def print_constants():
    output = "Size: "+str(global_size)+"\n"+"Degree: "+str(global_degree)+"\n"+"Max Weight: "+str(global_max_weight)+"\n"+"Max Read: "+str(global_max_read)+"\n"
    print(output)
 

In [ ]:
def random_graph(degree,size):
    graph = nx.generators.random_regular_graph(degree, size)
    return graph
    

In [ ]:
def expander_graph(size):
    graph = nx.generators.paley_graph(size)
    return graph


In [ ]:
def complete_graph(size):
    graph = nx.generators.complete_graph(size)
    return graph

In [ ]:
def cycle_graph(size):
    graph = nx.generators.cycle_graph(size)
    return graph


In [ ]:
def assign_random_edge_weights(graph, max_weight):
    for (u, v) in graph.edges():
        graph.edges[u, v]['weight'] = random.randint(1, max_weight)
    return graph

def assign_exponantion_edge_weights(graph):
    for (u, v) in graph.edges():
        graph.edges[u, v]['weight'] = round(np.random.exponantial())
    return graph

def assign_normal_edge_weights(graph, max_weight):
    for (u, v) in graph.edges():
        graph.edges[u, v]['weight'] = round(np.random.normal(loc=(max_weight/2)))
    return graph

In [ ]:
def kruskall_comparison():
    print("Create graph with the following constants:")
    print_constants()
    
    #Chose here different graph options from above
    #graph = random_graph(global_degree,global_size)
    graph = complete_graph(global_size)

    #Chose here different weight sampling options from above
    graph = assign_random_edge_weights(graph,global_max_weight)
    
    #Our algorithm
    our_start_time = time.time()
    approx = MST(
        NetworkXGraph(graph, global_size, global_max_weight, global_max_read)
    ).approx_weight()
    our_runtime = time.time() - our_start_time
    print("Our Runtime: "+str(our_runtime))
    print("Our MST weight approx. for random graph:", approx,"\n")
    
    #NetworkX Kruskall Algorithm
    kruskall_start_time = time.time()
    mst=nx.minimum_spanning_tree(graph)
    kruskall_time = time.time()-kruskall_start_time
    print("NetworkX Kruskall runtime: "+str(kruskall_time))

    gt_w = sum(graph.edges[e]["weight"] for e in mst.edges)
    print("ground truth weight", gt_w)

    ratio = max(gt_w, approx) / min(gt_w, approx)
    print("error %", 100 * (ratio - 1))
    

In [ ]:
kruskall_comparison()